In [6]:
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels
import matplotlib.pyplot as plt
import dodiscover as dod
import hyppo
import scipy as sp
import sklearn as sk
from sims import *
import dask
from dask.distributed import Client, progress
import dask.dataframe as ddf
import logging
import warnings
warnings.filterwarnings("ignore")
import contextlib

# Simulations

In [7]:
nbreaks = 2
nsamp = 100
plow = 10
phigh = 101
bhigh = 0.8
blow = 0.4
K = 3
nsims = 2
ncores = 50
Nrep = 2
covar_sz = 1

def run_row(row, nrep=Nrep):
    fns_to_run = {"cMANOVA": cond_manova, "CoDITE" : codite, "cDCorr" : cond_dcorr, "DCorr" : dcorr}
    sim_fn = simulations[row["Setting"]]
    pvals = []
    stat_names = []
    if row["Setting"] == "K-Class":
        Y, T, X, _, _, _ = sim_fn(row["#Samples"], row["Dimensionality"],
                                  causal_effect_size=row["Effect Size"], covar_effect_size=covar_sz,
                                  balance=row["Balance"],
                                  K=3)
    else:
        Y, T, X, _, _, _ = sim_fn(row["#Samples"], row["Dimensionality"],
                                  causal_effect_size=row["Effect Size"], covar_effect_size=covar_sz,
                                  balance=row["Balance"])
    for statname, fn in fns_to_run.items():
        try:
            pval, _ = fn(Y, T, X, nrep=nrep)
            pvals.append(pval)
            stat_names.append(statname)
        except:
            pvals.append(float("NaN"))
    with contextlib.redirect_stdout(None):
        balanced_ids = causal_prep(X, T)
    X_bal = X[balanced_ids]; T_bal = T[balanced_ids]; Y_bal = Y[balanced_ids,:]
    for statname, fn in fns_to_run.items():
        try:
            pval, _ = fn(Y_bal, T_bal, X_bal, nrep=nrep)
            pvals.append(pval)
            stat_names.append("Causal {:s}".format(statname))
        except:
            pvals.append(float("NaN"))
    return tuple([row["Setting"], row["#Samples"], row["Dimensionality"], row["Balance"],
                  row["Effect Size"], row["i"], *pvals])

simulations = {
    "Linear" : linear_sim,
    "Sigmoidal": sigmoidal_sim,
    "K-Class": kclass_sim
}

exps = []
for sim in simulations.keys():
    print(sim)
    for p in [plow, phigh]:
        for balance in [blow, bhigh]:
            for eff_sz in np.linspace(0, 2, nbreaks):
                for i in range(nsims):
                    exps.append([sim, nsamp, p, balance, eff_sz, i])

sim_exps = pd.DataFrame(exps, columns=["Setting", "#Samples", "Dimensionality", "Balance", "Effect Size", "i"])
print(sim_exps.head(n=10))
print(sim_exps.shape)

Linear
Sigmoidal
K-Class
  Setting  #Samples  Dimensionality  Balance  Effect Size  i
0  Linear       100              10      0.4          0.0  0
1  Linear       100              10      0.4          0.0  1
2  Linear       100              10      0.4          2.0  0
3  Linear       100              10      0.4          2.0  1
4  Linear       100              10      0.8          0.0  0
5  Linear       100              10      0.8          0.0  1
6  Linear       100              10      0.8          2.0  0
7  Linear       100              10      0.8          2.0  1
8  Linear       100             101      0.4          0.0  0
9  Linear       100             101      0.4          0.0  1
(48, 6)


In [3]:
client = Client(threads_per_worker=1, n_workers=ncores, silence_logs=logging.ERROR)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 50
Total threads: 50,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51287,Workers: 50
Dashboard: http://127.0.0.1:8787/status,Total threads: 50
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:51390,Total threads: 1
Dashboard: http://127.0.0.1:51393/status,Memory: 655.36 MiB
Nanny: tcp://127.0.0.1:51290,


In [8]:
sim_exps = ddf.from_pandas(sim_exps, npartitions=ncores)
sim_results = sim_exps.apply(lambda x: run_row(x), axis=1, result_type='expand',
                             meta={0: str, 1: int, 2: int, 3: float, 4: float,
                                   5: int, 6: float, 7: float, 8: float, 9: float, 10: float, 11: float,
                                   12: float, 13: float})

In [9]:
sim_results = sim_results.compute(scheduler="multiprocessing")

/Users/eric/.virtualenvs/causal/lib/python3.10/site-packages/hyppo/tools/common.py:92: RuntimeWarning: The number of replications is low (under 1000), and p-value calculations may be unreliable. Use the p-value result, with caution!
  warnings.warn(msg, RuntimeWarning)
/Users/eric/.virtualenvs/causal/lib/python3.10/site-packages/hyppo/tools/common.py:92: RuntimeWarning: The number of replications is low (under 1000), and p-value calculations may be unreliable. Use the p-value result, with caution!
  warnings.warn(msg, RuntimeWarning)
/Users/eric/.virtualenvs/causal/lib/python3.10/site-packages/hyppo/tools/common.py:92: RuntimeWarning: The number of replications is low (under 1000), and p-value calculations may be unreliable. Use the p-value result, with caution!
  warnings.warn(msg, RuntimeWarning)
/Users/eric/.virtualenvs/causal/lib/python3.10/site-packages/hyppo/tools/common.py:92: RuntimeWarning: The number of replications is low (under 1000), and p-value calculations may be unrelia

In [ ]:
sim_results = sim_results.rename(columns={0: "Simulation", 1: "#Samples", 2: "Dimensionality", 3: "Balance",
                                          4: "Effect Size", 5: "i", 6: "cMANOVA", 7: "CoDITE", 8: "cDCorr",
                                          9: "DCorr", 10: "Causal MANOVA", 11: "Causal DITE", 12: "Causal cDCorr",
                                          13: "Causal DCorr"})
sim_results.to_pickle('./data/sim_results.pkl')

In [ ]:
sim_results